In [192]:
import boto3
import pandas as pd
from datetime import datetime, timedelta
import os
import io
from collections import Counter
import re
import numpy as np

service_name = 's3'
endpoint_url = 'https://kr.object.ncloudstorage.com'
access_key = ' '
secret_key = ' '
bucket_name = 'bucket1chaegpt'

In [193]:
date = pd.read_excel('C:/Users/01061.DESKTOP-UR6KUEV/KT_visang/crge_차트분석.xlsx', index_col=0)
date.reset_index(inplace=True, drop=True)
# tick = date.Ticker[0]
date

,Start Date,End Date,Duration,Change Average
0,2023-08-23,2023-08-24,2,8.859650
1,2023-05-09,2023-05-15,5,8.850792
2,2023-04-28,2023-05-04,5,-7.597213
3,2023-04-26,2023-04-27,2,7.419180


In [194]:
#df 피클 파일 불러오기
df = pd.read_pickle('미국뉴스_요약감성키워드.pickle')
df = df[df['Ticker']=='CRGE']
df

,검색어,기사종류,제목,발행일,기사내용,링크,Ticker,sum_text,감성분석,word_list
2639,Charge Enterprises,Insight,Insight: China quietly recruits overseas chip ...,2023-08-25,"For a decade until 2018, China sought to recr...",https://www.reuters.com/technology/china-quiet...,CRGE,"Qiming program offers home-purchase subsidies,...","[{'label': 'neutral', 'score': 0.9996109604835...","[China, Thousand, Tal, ents, Plan, China, sign..."
2640,Charge Enterprises,Legal,Trump Georgia indictment: RICO charges could b...,2023-08-17,A Georgia state law against racketeering coul...,https://www.reuters.com/legal/trump-georgia-in...,CRGE,District Attorney Fani Willis says case is pol...,"[{'label': 'neutral', 'score': 0.9997786879539...","[Georgia, state, law, racket, eering, organize..."
2641,Charge Enterprises,Disrupted,"Microsoft to charge more for AI in office, sec...",2023-07-19,Microsoft (MSFT.O) on Tuesday said it would c...,https://www.reuters.com/technology/microsoft-c...,CRGE,"Customers to pay $30 per user, per month. The ...","[{'label': 'neutral', 'score': 0.9972696900367...","[Microsoft, artificial, intelligence, office, ..."
2642,Charge Enterprises,Business,Comcast president Cavanagh shakes up NBCUniver...,2023-07-07,Comcast's (CMCSA.O) Mike Cavanagh named Unive...,https://www.reuters.com/business/media-telecom...,CRGE,Langley named chairman of NBCUniversal Studio ...,"[{'label': 'neutral', 'score': 0.99957674741745}]","[NBC, Universal, Studio, Group, chief, content..."
2643,Charge Enterprises,Breakingviews,Adani case threatens watchdog’s rising credibi...,2023-07-06,India’s $3.6 trillion equity market is coming...,https://www.reuters.com/breakingviews/adani-ca...,CRGE,Markets regulator Madhabi Puri Buch has built ...,"[{'label': 'negative', 'score': 0.992322146892...","[India, Securities, and, Exchange, Board, Indi..."
2644,Charge Enterprises,Markets,China's JD.com to open grocery stores by mergi...,2023-06-26,Chinese e-commerce giant JD.com (9618.HK) pla...,https://www.reuters.com/markets/deals/chinas-j...,CRGE,Merger of 7Fresh with other business lines see...,"[{'label': 'neutral', 'score': 0.9995597004890...","[retail, grocery, stores, 7, Fresh, Pin, pin, ..."
2645,Charge Enterprises,Sports,Club by club review of the Premier League season,2023-05-29,Manchester City won their third straight Prem...,https://www.reuters.com/sports/soccer/club-by-...,CRGE,Manchester City retained their Premier League ...,"[{'label': 'neutral', 'score': 0.9984790682792...","[Manchester, City, Premier, League, Arsenal, C..."
2646,Charge Enterprises,World,UK's Workspace forecasts rental growth after p...,2023-05-25,Workspace Group (WKP.L) forecast rental incom...,https://www.reuters.com/world/uk/workspace-sli...,CRGE,London-focused office-space provider posts ann...,"[{'label': 'positive', 'score': 0.999504208564...","[Works, pace, Group, rental, income, pricing, ..."
2647,Charge Enterprises,Markets,"U.S. prosecutor query, quarterly loss hit alre...",2023-05-11,Billionaire Carl Icahn's holding company said...,https://www.reuters.com/markets/us/icahn-enter...,CRGE,IEP shares have lost 40% of their value since ...,"[{'label': 'negative', 'score': 0.978486299514...","[holding, company, Hind, enburg, Research, fed..."
2648,Charge Enterprises,Breakingviews,ValueAct loses its temper in 7-Eleven,2023-05-10,ValueAct Capital’s chief Mason Morfit prefers...,https://www.reuters.com/breakingviews/valueact...,CRGE,Activist says 7-Eleven could be worth $40 bill...,"[{'label': 'neutral', 'score': 0.9997804760932...","[Value, Act, Capital, PowerPoint, flame, war, ..."


In [195]:
date['키워드'] = ''
date['키워드 등장 시점'] = ''
date['요약'] = ''
date['링크'] = ''
for i in range(len(date)):
    start_date = pd.to_datetime(date.iloc[i,0])
    end_date = pd.to_datetime(date.iloc[i,1])

    df['발행일'] = pd.to_datetime(df['발행일'])

    filtered_df = df[(df['발행일'] >= start_date) & (df['발행일'] <= end_date)]

    # 키워드 추출 및 빈도 계산
    all_keywords = []
    for word_list in filtered_df['word_list']:
        all_keywords.extend(word_list)

    keyword_counts = Counter(all_keywords)

    # 상위 3개 키워드 추출
    top_keywords = keyword_counts.most_common(3)
    keywords_only = [keyword for keyword, _ in top_keywords]
    if keywords_only != []:
        first_keyword = keywords_only[0]

        filtered_dates = filtered_df[filtered_df['word_list'].apply(lambda x: first_keyword in x)]['발행일']
        # 가장 오래된 발행일 찾기
        oldest_date = min(filtered_dates)

        # 가장 오래된 발행일에 해당하는 sum_text 가져오기
        oldest_sum_text = filtered_df[filtered_df['발행일'] == oldest_date]['sum_text'].values[0]

        # 가장 오래된 발행일에 해당하는 링크 가져오기
        oldest_link = filtered_df[filtered_df['발행일'] == oldest_date]['링크'].values[0]

        date.at[i, '키워드']  = keywords_only
        date.at[i, '요약']  = oldest_sum_text
        date.at[i, '키워드 등장 시점'] = oldest_date
        date.at[i, '링크']  = oldest_link

In [196]:
date

,Start Date,End Date,Duration,Change Average,키워드,키워드 등장 시점,요약,링크
0,2023-08-23,2023-08-24,2,8.859650,,,,
1,2023-05-09,2023-05-15,5,8.850792,"[Value, Act, Research]",2023-05-10 00:00:00,Activist says 7-Eleven could be worth $40 bill...,https://www.reuters.com/breakingviews/valueact...
2,2023-04-28,2023-05-04,5,-7.597213,,,,
3,2023-04-26,2023-04-27,2,7.419180,,,,
